<a href="https://colab.research.google.com/github/charlaburnett/change_detection_ukraine/blob/main/clean_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive')

from credentials import config


In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='gee-charlaburnett')

In [ ]:
import ee

mariupol = ee.Geometry.Rectangle([37.4, 46.8, 37.7, 47.2])

pre_start = '2021-01-01'
pre_end = '2021-03-01'
post_start = '2022-05-01'
post_end = '2022-07-01'

In [ ]:
def get_s1_vv(start, end):
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(mariupol)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .select('VV')
            .mean())


pre_image = get_s1_vv(pre_start, pre_end)
post_image = get_s1_vv(post_start, post_end)


In [ ]:
change_image = post_image.subtract(pre_image)

In [ ]:
import geemap

Map = geemap.Map(center = [47.1, 37.55], zoom= 11)

vis_params = {
    'min': -2,
    'max': 2,
    'palette': ['blue', 'white', 'red']
}

Map.addLayer(change_image, vis_params, 'SAR Change Detection')
Map.addLayerControl()
Map

Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
change_red_only = change_image.updateMask(change_image.gt(1.0))
red_vis = {
    'min': 0.5,
    'max': 2,
    'palette': ['red']
}

In [ ]:
Map_red = geemap.Map(center=[47.1, 37.55], zoom=11, basemap='SATELLITE')

Map_red.addLayer(change_red_only, red_vis, 'Red-only Positive Change')
Map_red.addLayerControl()
Map_red

Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
# Remove Watermarks

landcover = ee.Image("ESA/WorldCover/v100/2020")
non_water_mask = landcover.neq(80)

change_red_only_clean = change_red_only.updateMask(non_water_mask)


In [ ]:
Map_clean = geemap.Map(center = [47.1, 37.55], zoom= 11, basemap ='Esri.WorldImagery')
Map_clean.addLayer(change_red_only_clean, red_vis, 'Cleaned Red Change')
Map_clean.addLayerControl()
Map_clean

Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
## Pixel Area Changes

stats = change_red_only_clean.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol,
    scale = 10,
    maxPixels=1e9
)

area_sq_m = stats.getInfo() ['VV']
area_sq_km = area_sq_m/ 1e6

print(f"Total change area: {area_sq_km:.2f} sq km")


Total change area: 147.20 sq km


In [ ]:
#Focus on Urban Areas

landcover = ee.Image("ESA/WorldCover/v100/2020")

urban_mask = landcover.eq(50)

change_urban_only = change_red_only.updateMask(urban_mask)

In [ ]:
Map_urban = geemap.Map(center=[47.1, 37.55], zoom=11, basemap='Esri.WorldImagery')

Map_urban.addLayer(change_urban_only, red_vis, 'Urban-only SAR Change')
Map_urban.addLayerControl()
Map_urban


Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [ ]:
stats = change_urban_only.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol,
    scale=10,
    maxPixels=1e9
)

area_sq_m = stats.getInfo()['VV']
area_sq_km = area_sq_m / 1e6
print(f"Urban change area: {area_sq_km:.2f} sq km ")

Urban change area: 43.26 sq km 


In [ ]:
total_area_m2 = ee.Image.pixelArea().reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol,
    scale=10,
    maxPixels=1e9,
).getInfo()['area']

total_area_km2 = total_area_m2 /1e6

In [ ]:
percent_changed = (area_sq_km / total_area_km2) * 100
print(f"Urban change area: {area_sq_km:.2f} sq km")
print(f"That's approximately {percent_changed:.1f}% of the AOI ({total_area_km2:.2f} sq km).")

Urban change area: 43.26 sq km
That's approximately 4.3% of the AOI (1014.62 sq km).


In [ ]:
 ghsl = ee.Image("JRC/GHSL/P2016/BUILT_LDSMT_GLOBE_V1") \
    .select('built') \
    .gt(0)  # Pixels with any built-up value


In [ ]:

change_area_stats = change_urban_builtup.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol_aoi,
    scale=10,
    maxPixels=1e9
)

change_km2 = change_area_stats.getInfo()['VV'] / 1e6

ghsl_mask_named = ghsl.updateMask(ghsl).rename('built')


urban_area_stats = ghsl_mask_named.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol_aoi,
    scale=10,
    maxPixels=1e9
)

urban_km2 = urban_area_stats.getInfo()['built'] / 1e6

percent_change = (change_km2 / urban_km2) * 100

print(f"Urban SAR change: {change_km2:.2f} sq km")
print(f"Total built-up area: {urban_km2:.2f} sq km")
print(f"That's {percent_change:.1f}% of built-up Mariupol.")



Urban SAR change: 64.89 sq km
Total built-up area: 172.13 sq km
That's 37.7% of built-up Mariupol.


In [ ]:
!jupyter nbconvert \
  --ClearMetadataPreprocessor.enabled=True \
  --ClearOutputPreprocessor.enabled=True \
  --to notebook \
  --output clean_notebook.ipynb \
  change-detection-ukraine.ipynb

[NbConvertApp] WARNING | pattern 'change-detection-ukraine.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]


In [ ]:
import os

for f in os.listdir():
    if f.endswith('.ipynb'):
        print(f)
